In [1]:

import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from scipy.stats import mannwhitneyu, ttest_ind

In [2]:
adata = ad.read_h5ad("Geneformer_pretrained/adata_05a_with_embeddings.h5ad")

In [3]:
adata 

AnnData object with n_obs × n_vars = 49641 × 27187
    obs: 'prject', 'nUMI', 'nGene', 'cellID', 'Doublet_Score', 'Doublet_Class', 'log10GenesPerUMI', 'mitoRatio', 'riboRatio1', 'riboRatio2', 'riboRatio3', 'riboRatio4', 'riboRatio', 'RNA_snn_res.0', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'class1', 'sample', 'source', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.1.2', 'RNA_snn_res.1.4', 'RNA_snn_res.1.6', 'RNA_snn_res.1.8', 'RNA_snn_res.2', 'cellType', 'dcellType', 'cell', 'patient', 'timing', 'category', 'response', 'sub.response', 'date', 'start', 'daysSinceStart', 'blast', 'age', 'sex', 'FAB', 'cellCount', 'monthsSinceStart', 'relapse', 'daysStartRelapse', 'monthsStartRelapse', 'azimuthNames', 'mut', 'p53.mut', 'n_counts', 'venetoclax', 'treat'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.varianc

In [16]:
adata.obs[ 'venetoclax'].value_counts()

venetoclax
nonresponder    16609
responder       10351
Name: count, dtype: int64

In [4]:
adata.obs['dcellType'].value_counts()

dcellType
CD4               14678
Late Erythroid     6026
NK                 4155
CD8 CTL            4113
CD14 Mono          3773
B cells            3219
BC2                2364
CD8 Mem            1710
CD8 Naive          1570
CD8 Ex             1281
BC3                1103
DC                  905
BC1                 884
BC4                 883
GMP                 816
CD8 EM              709
pre B               492
CD16 Mono           292
Macrophage          253
Prog Mk             247
Plasma              168
Name: count, dtype: int64

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu, ttest_ind


def compute_celltype_fraction_select(celltype, adata, *, 
                                     treats=None, ven_states=None, response_eq=None,
                                     label_treat=None, label_ven=None):
    """
    通用选择器版本：
    - treats: None / str / list[str]   (e.g., 'pre', 'post', ['pre','post'])
    - ven_states: None / str / list[str]  (e.g., 'responder', 'nonresponder', ['responder','nonresponder'])
    - response_eq: None / 0 / 1  (直接使用 adata.obs['response'] == response_eq 进行过滤; 健康对照可用 0)
    - label_treat, label_ven: 用于输出标签（不传则自动从条件生成）
    """
    # ---- 构造筛选 mask ----
    obs = adata.obs
    mask = pd.Series(True, index=obs.index)

    # 用 response_eq 直接筛（健康=0，患者=1）
    if response_eq is not None:
        if 'response' not in obs.columns:
            raise KeyError("adata.obs 中没有 'response' 列。")
        mask &= (obs['response'] == response_eq)

    # treats
    if treats is not None:
        treats_list = [treats] if isinstance(treats, str) else list(treats)
        mask &= obs['treat'].isin(treats_list)
        if label_treat is None:
            label_treat = "+".join(treats_list)
    else:
        if label_treat is None:
            label_treat = "all_treats"

    # venetoclax / responder状态
    if ven_states is not None:
        ven_list = [ven_states] if isinstance(ven_states, str) else list(ven_states)
        mask &= obs['venetoclax'].isin(ven_list)
        if label_ven is None:
            label_ven = "+".join(ven_list)
    else:
        if label_ven is None:
            label_ven = "all_states"

    subset = obs[mask]

    def celltype_fraction(df):
        total = len(df)
        n_celltype = (df["dcellType"].str.startswith(celltype)).sum()
        frac = n_celltype / total if total > 0 else np.nan
        return pd.Series({"fraction": frac, "count": n_celltype, "total": total})

    # 每个 sample 统计
    fractions = subset.groupby("sample", observed=True).apply(
        celltype_fraction, include_groups=False
    )

    return {
        "treat": label_treat,
        "venetoclax": label_ven,
        "n_samples": fractions.shape[0],
        "mean_fraction": fractions["fraction"].mean(),
        "std_fraction": fractions["fraction"].std(),
        "sem_fraction": fractions["fraction"].sem(),
        "mean_count": fractions["count"].mean(),
        "fractions": fractions
    }

# ======== 组别组合 ========
celltype = "CD8"
results = []

# 原来的四组
results.append(compute_celltype_fraction_select(celltype, adata, treats='pre',  ven_states='responder'))
results.append(compute_celltype_fraction_select(celltype, adata, treats='post', ven_states='responder'))
results.append(compute_celltype_fraction_select(celltype, adata, treats='pre',  ven_states='nonresponder'))
results.append(compute_celltype_fraction_select(celltype, adata, treats='post', ven_states='nonresponder'))

# pre+post（分别对 responder / nonresponder）
results.append(compute_celltype_fraction_select(celltype, adata, treats=['pre','post'], ven_states='responder',     label_treat='pre+post'))
results.append(compute_celltype_fraction_select(celltype, adata, treats=['pre','post'], ven_states='nonresponder',  label_treat='pre+post'))

# pre+post 对应 response+nonresponse（即所有 AML 合并）
results.append(compute_celltype_fraction_select(celltype, adata, treats=['pre','post'], ven_states=['responder','nonresponder'],
                                                label_treat='pre+post', label_ven='responder+nonresponder'))

# 健康对照（response==0），不限定 treat / venetoclax
results.append(compute_celltype_fraction_select(celltype, adata, response_eq=0, 
                                                label_treat='healthy', label_ven='control'))

# —— 三位有效数字的显示工具 —— 
def format_sig(x, sig=3):
    return f"{x:.{sig}g}" if pd.notnull(x) else x

# 汇总表（带 details）
summary = pd.DataFrame([
    {
        "treat": r["treat"],
        "venetoclax": r["venetoclax"],
        "n_samples": r["n_samples"],
        "mean": format_sig(r["mean_fraction"], 3),
        "std": format_sig(r["std_fraction"], 3),
        "sem": format_sig(r["sem_fraction"], 3),
        "details": "; ".join(
            f"{idx}: count={int(row['count'])}, fraction={format_sig(row['fraction'], 3)}, total={int(row['total'])}"
            for idx, row in r["fractions"].iterrows()
        )
    }
    for r in results
])

print(summary.to_string(index=False))
summary.to_csv(f'{celltype}_with_controls.csv', index=False)



# ========== 上一步得到的 results 和 summary 已有 ==========

# 拉成长表
long_rows = []
for r in results:
    tmp = r["fractions"].reset_index().rename(columns={"index":"sample"})
    tmp["treat"] = r["treat"]
    tmp["venetoclax"] = r["venetoclax"]
    long_rows.append(tmp)
long_df = pd.concat(long_rows, ignore_index=True)
# long_df 包含：sample, fraction, count, total, treat, venetoclax

# 定义检验函数
def test_two_groups(df, g1, g2, method="mannwhitney"):
    a = df[(df["treat"]==g1[0]) & (df["venetoclax"]==g1[1])]["fraction"].dropna().values
    b = df[(df["treat"]==g2[0]) & (df["venetoclax"]==g2[1])]["fraction"].dropna().values
    label1 = f"{g1[0]}-{g1[1]}"
    label2 = f"{g2[0]}-{g2[1]}"

    if len(a)==0 or len(b)==0:
        return {"group1": label1, "group2": label2, "method": method,
                "n1": len(a), "n2": len(b),
                "mean1": np.nan, "mean2": np.nan,
                "median1": np.nan, "median2": np.nan,
                "stat": np.nan, "p": np.nan}

    if method=="mannwhitney":
        stat, p = mannwhitneyu(a, b, alternative="two-sided")
        m = "Mann-Whitney U"
    else:
        stat, p = ttest_ind(a, b, equal_var=False)
        m = "Welch t-test"

    return {"group1": label1, "group2": label2, "method": m,
            "n1": len(a), "n2": len(b),
            "mean1": np.mean(a), "mean2": np.mean(b),
            "median1": np.median(a), "median2": np.median(b),
            "stat": stat, "p": p}

# 多重比较校正（Benjamini–Hochberg）
def fdr_bh(p):
    p = np.asarray(p, dtype=float)
    n = p.size
    order = np.argsort(p)
    ranked = np.empty_like(order)
    ranked[order] = np.arange(1, n+1)
    q = p * n / ranked
    q_sorted = np.minimum.accumulate(q[order][::-1])[::-1]
    q_adj = np.empty_like(q)
    q_adj[order] = np.minimum(q_sorted, 1.0)
    return q_adj

# 需要比较的组合对，可以按需求扩展
pairs = [
    (("pre","responder"), ("post","responder")),
    (("pre","nonresponder"), ("post","nonresponder")),
    (("pre","responder"), ("pre","nonresponder")),
    (("post","responder"), ("post","nonresponder")),
    (("pre+post","responder"), ("pre+post","nonresponder")),
    (("pre","responder"), ("healthy","control")),  # AML 合并 vs 健康
    (("pre","nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
    (("post","responder"), ("healthy","control")),  # AML 合并 vs 健康
    (("post","nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
    (("pre+post","responder"), ("healthy","control")),  # AML 合并 vs 健康
    (("pre+post","nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
    (("pre+post","responder+nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
]

rows = []
for g1,g2 in pairs:
    rows.append(test_two_groups(long_df, g1, g2, method="mannwhitney"))
    rows.append(test_two_groups(long_df, g1, g2, method="ttest"))

comp = pd.DataFrame(rows)

# FDR 校正
comp["p_adj_BH"] = fdr_bh(comp["p"].fillna(1.0).values)

# 三位有效数字
def format_sig(x, sig=3):
    return f"{x:.{sig}g}" if pd.notnull(x) else x

for col in ["mean1","mean2","median1","median2","stat","p","p_adj_BH"]:
    comp[col] = comp[col].apply(format_sig)

print("\nPairwise comparisons:")
print(comp[['group1','group2','method','p']].to_string(index=False))

comp[['group1','group2','method','p']].to_csv(f'{celltype}_pairwise_with_healthy.csv', index=False)


In [16]:
import gseapy as gp
celltype = "CD8"
pairs = [
    (("pre","responder"), ("post","responder")),
    (("pre","nonresponder"), ("post","nonresponder")),
    (("pre","responder"), ("pre","nonresponder")),
    (("post","responder"), ("post","nonresponder")),
    # (("pre+post","responder"), ("pre+post","nonresponder")),
    # (("pre","responder"), ("healthy","control")),  # AML 合并 vs 健康
    # (("pre","nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
    # (("post","responder"), ("healthy","control")),  # AML 合并 vs 健康
    # (("post","nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
    # (("pre+post","responder"), ("healthy","control")),  # AML 合并 vs 健康
    # (("pre+post","nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
    # (("pre+post","responder+nonresponder"), ("healthy","control")),  # AML 合并 vs 健康
]

def return_subset_mask(celltype, adata, *, treats=None, ven_states=None, response_eq=None, label_treat=None, label_ven=None):
    """
    通用选择器版本：
    - treats: None / str / list[str]   (e.g., 'pre', 'post', ['pre','post'])
    - ven_states: None / str / list[str]  (e.g., 'responder', 'nonresponder', ['responder','nonresponder'])
    - response_eq: None / 0 / 1  (直接使用 adata.obs['response'] == response_eq 进行过滤; 健康对照可用 0)
    - label_treat, label_ven: 用于输出标签（不传则自动从条件生成）
    """
    # ---- 构造筛选 mask ----
    obs = adata.obs
    mask = pd.Series(True, index=obs.index)

    # 用 response_eq 直接筛（健康=0，患者=1）
    if response_eq is not None:
        if 'response' not in obs.columns:
            raise KeyError("adata.obs 中没有 'response' 列。")
        mask &= (obs['response'] == response_eq)

    # treats
    if treats is not None:
        treats_list = [treats] if isinstance(treats, str) else list(treats)
        mask &= obs['treat'].isin(treats_list)
        if label_treat is None:
            label_treat = "+".join(treats_list)
    else:
        if label_treat is None:
            label_treat = "all_treats"

    # venetoclax / responder状态
    if ven_states is not None:
        ven_list = [ven_states] if isinstance(ven_states, str) else list(ven_states)
        mask &= obs['venetoclax'].isin(ven_list)
        if label_ven is None:
            label_ven = "+".join(ven_list)
    else:
        if label_ven is None:
            label_ven = "all_states"

    # dcellType 前缀筛选
    if celltype is not None:
        mask &= obs["dcellType"].str.startswith(celltype)

    # subset = adata[mask, :].X

    return mask


In [15]:


def run_go_analysis(genes, background=None, database="GO_Biological_Process_2021"):
    enr = gp.enrichr(
        gene_list=genes,
        gene_sets=database,
        background=background,   # 如果不传，会自动用数据库背景
        organism="Human",        # 如果是小鼠，写 "Mouse"
        outdir=None
    )
    return enr.results



def test_two_subsets(adata, mask1, mask2, method="mannwhitney"):
        
    adata.obs["subset"] = "other"
    adata.obs.loc[mask1, "subset"] = "group1"
    adata.obs.loc[mask2, "subset"] = "group2"

    # 差异分析
    sc.tl.rank_genes_groups(adata, "subset", groups=["group2"], reference="group1", method="wilcoxon")

    # 转成 pandas
    deg = sc.get.rank_genes_groups_df(adata, group="group1")

    # 筛选 FDR<0.05
    deg_sig = deg[deg["pvals_adj"] < 0.05]

    # 按显著性排序
    deg_sig = deg_sig.sort_values("pvals_adj")
    return deg_sig



for g1,g2 in pairs:
    mask1 = return_subset_mask(celltype, adata, treats=g1[0],  ven_states=g1[1])
    mask2 = return_subset_mask(celltype, adata, treats=g2[0],  ven_states=g2[1])

    deg_sig = test_two_subsets(adata, mask1, mask2)
    print(g1,g2, "显著差异基因数：", deg_sig.shape[0])
    
    # 生成文件名，例如：Erythroid_pre_responder_vs_post_nonresponder.csv
    fname = f"Genes_{celltype}_{g1[0]}_{g1[1]}_vs_{g2[0]}_{g2[1]}.csv"
    fname = fname.replace(" ", "_")
    deg_sig.to_csv(fname, index=False)

    if deg_sig.shape[0] > 0:
        genes = deg_sig["names"].tolist()
        go_res = run_go_analysis(genes)
        go_res_filtered = go_res[go_res['Genes'].apply(lambda x: len(x.split(';')) >= 5)]

        goname = f"GO_{celltype}_{g1[0]}_{g1[1]}_vs_{g2[0]}_{g2[1]}.csv"
        goname = goname.replace(" ", "_")
        go_res_filtered.to_csv(goname, index=False)

    #     print(go_res.head(10))   # 打印前10条 GO term


KeyboardInterrupt



In [19]:
def run_go_analysis(genes, background=None, database="GO_Biological_Process_2021"):
    enr = gp.enrichr(
        gene_list=genes,
        gene_sets=database,
        background=background,
        organism="Human",
        outdir=None
    )
    return enr.results


def test_two_subsets(adata, mask1, mask2, method="wilcoxon"):
    adata.obs["subset"] = "other"
    adata.obs.loc[mask1, "subset"] = "group1"
    adata.obs.loc[mask2, "subset"] = "group2"

    # 差异分析
    sc.tl.rank_genes_groups(adata, "subset", groups=["group1"], reference="group2", method=method)
    deg = sc.get.rank_genes_groups_df(adata, group="group1")

    # ⭐修改①：FDR<0.05（明确对应论文标准）
    # deg_sig = deg[deg["pvals_adj"] < 0.05].sort_values("pvals_adj")
    deg_sig = deg[(deg["pvals_adj"] < 0.05) & (abs(deg["logfoldchanges"]) > 1)]
    deg_sig = deg_sig.sort_values("pvals_adj")
    return deg_sig


# 主循环部分
for g1, g2 in pairs:
    mask1 = return_subset_mask(celltype, adata, treats=g1[0], ven_states=g1[1])
    mask2 = return_subset_mask(celltype, adata, treats=g2[0], ven_states=g2[1])

    deg_sig = test_two_subsets(adata, mask1, mask2)
    print(g1, g2, "显著差异基因数：", deg_sig.shape[0])

    # 生成文件名
    fname = f"Genes_{celltype}_{g1[0]}_{g1[1]}_vs_{g2[0]}_{g2[1]}.csv".replace(" ", "_")
    deg_sig.to_csv(fname, index=False, quotechar='"')

    if deg_sig.shape[0] == 0:
        continue

    # ⭐修改②：将差异基因分为上调/下调两组（新增逻辑）
    deg_up = deg_sig[deg_sig["logfoldchanges"] > 0]
    deg_down = deg_sig[deg_sig["logfoldchanges"] < 0]

    # ⭐修改③：分别对 up/down 做 GO 富集
    for direction, subset in [("Up", deg_up), ("Down", deg_down)]:
        if subset.shape[0] < 1:
            continue

        genes = subset["names"].tolist()
        go_res = run_go_analysis(genes)
        if "Overlap" in go_res.columns:
            go_res["Overlap"] = go_res["Overlap"].astype(str).apply(lambda x: f"'{x}")  

        # ⭐修改④：只保留包含 ≥5 基因的 GO term（调整 split 符号为 ';'）
        go_res_filtered = go_res[go_res['Genes'].apply(lambda x: len(x.split(';')) >= 5)]

        # ⭐修改⑤：输出文件名增加 Up/Down 标签
        goname = f"GO_{direction}_{celltype}_{g1[0]}_{g1[1]}_vs_{g2[0]}_{g2[1]}.csv".replace(" ", "_")
        go_res_filtered.to_csv(goname, index=False, quotechar='"')


('pre', 'responder') ('post', 'responder') 显著差异基因数： 230
('pre', 'nonresponder') ('post', 'nonresponder') 显著差异基因数： 291
('pre', 'responder') ('pre', 'nonresponder') 显著差异基因数： 162
('post', 'responder') ('post', 'nonresponder') 显著差异基因数： 182


In [ ]:

    """
    根据两组细胞筛选基因：
    - 任一组里表达细胞占比 ≥ min_pct
    - 且任一组均值 ≥ min_mean（可设为0跳过）
    返回满足条件的 var_names 列表
    在差异分析前进行低表达基因过滤：
    - min_pct: 至少在某一组 ≥该比例的细胞中有非零表达（默认10%）
    - min_mean: 至少在某一组的平均表达量达到该阈值（默认0）
    - layer_for_filter: 若你的原始计数在某个layer（如 'counts'），可指定用于筛选
    """

In [24]:

import numpy as np
import pandas as pd
import scanpy as sc
import gseapy as gp
from scipy import sparse

def _get_matrix(adata, layer=None):
    return adata.layers[layer] if (layer is not None and layer in adata.layers) else adata.X

def _to_bool_np(x):
    return np.asarray(x, dtype=bool).ravel()

def _pct_mean_by_mask(adata, mask, layer=None):
    X = _get_matrix(adata, layer)
    Xsub = X[_to_bool_np(mask), :]
    if sparse.issparse(Xsub):
        pct = np.asarray((Xsub > 0).mean(axis=0)).ravel()
        mean = np.asarray(Xsub.mean(axis=0)).ravel()
    else:
        pct = np.asarray((Xsub > 0).mean(axis=0)).ravel()
        mean = np.asarray(Xsub.mean(axis=0)).ravel()
    return pct, mean  # shape (n_genes,)

def detectable_genes(adata, mask1, mask2, min_pct=0.1, min_mean=0.0, layer=None):
    pct1, mean1 = _pct_mean_by_mask(adata, mask1, layer)
    pct2, mean2 = _pct_mean_by_mask(adata, mask2, layer)
    keep = ((pct1 >= min_pct) | (pct2 >= min_pct)) & ((mean1 >= min_mean) | (mean2 >= min_mean))
    return adata.var_names[np.asarray(keep, dtype=bool)].tolist(), pct1, pct2

def run_go_analysis(genes, background=None, database="GO_Biological_Process_2021"):
    enr = gp.enrichr(
        gene_list=genes,
        gene_sets=database,
        background=background,
        organism="Human",
        outdir=None
    )
    return enr.results

def test_two_subsets_postfilter(
    adata, mask1, mask2, method="wilcoxon",
    min_pct=0.10, min_mean=0.0, layer_for_filter=None
):
    # 1) 打标签
    adata.obs["subset"] = "other"
    adata.obs.loc[_to_bool_np(mask1), "subset"] = "group1"
    adata.obs.loc[_to_bool_np(mask2), "subset"] = "group2"

    # 2) 全基因做差异分析（FDR 基数 = 全部基因）
    sc.tl.rank_genes_groups(adata, "subset", groups=["group1"], reference="group2", method=method)
    deg_full = sc.get.rank_genes_groups_df(adata, group="group1")

    # 3) 计算可检测基因（用于 post-filter 和 GO 背景）
    bg_genes, pct1, pct2 = detectable_genes(
        adata, mask1, mask2, min_pct=min_pct, min_mean=min_mean, layer=layer_for_filter
    )
    # 把占比并到结果里（便于后续观察/筛选）
    deg_full = deg_full.set_index("names")
    deg_full["pct_group1"] = pct1
    deg_full["pct_group2"] = pct2
    deg_full.reset_index(inplace=True)

    # 4) post-filter：仅保留“可检测基因” + 你的阈值
    deg_sig = deg_full[
        (deg_full["names"].isin(bg_genes)) &
        (deg_full["pvals_adj"] < 0.05) &
        (np.abs(deg_full["logfoldchanges"]) > 1)
    ].sort_values("pvals_adj")

    return deg_sig, bg_genes

# ================= 主循环 =================
for g1, g2 in pairs:
    mask1 = return_subset_mask(celltype, adata, treats=g1[0], ven_states=g1[1])
    mask2 = return_subset_mask(celltype, adata, treats=g2[0], ven_states=g2[1])

    deg_sig, bg_genes = test_two_subsets_postfilter(
        adata, mask1, mask2,
        method="wilcoxon",
        min_pct=0.10,          # 只统计/过滤时使用
        min_mean=0.0,
        layer_for_filter=None  # 如果有 counts：设为 "counts"
    )

    print(g1, g2, "显著差异基因数：", deg_sig.shape[0])

    fname = f"Genes_{celltype}_{g1[0]}_{g1[1]}_vs_{g2[0]}_{g2[1]}.csv".replace(" ", "_")
    deg_sig.to_csv(fname, index=False, quotechar='"')

    if deg_sig.shape[0] == 0:
        continue

    deg_up = deg_sig[deg_sig["logfoldchanges"] > 0]
    deg_down = deg_sig[deg_sig["logfoldchanges"] < 0]

    for direction, subset in [("Up", deg_up), ("Down", deg_down)]:
        if subset.shape[0] < 1:
            continue
        genes = subset["names"].tolist()
        go_res = run_go_analysis(genes, background=bg_genes)
        if "Overlap" in go_res.columns:
            go_res["Overlap"] = go_res["Overlap"].astype(str).apply(lambda x: f"'{x}")
        if "Genes" in go_res.columns:
            go_res = go_res[go_res["Genes"].apply(lambda x: len(str(x).split(";")) >= 5)]
        goname = f"GO_{direction}_{celltype}_{g1[0]}_{g1[1]}_vs_{g2[0]}_{g2[1]}.csv".replace(" ", "_")
        go_res.to_csv(goname, index=False, quotechar='"')


('pre', 'responder') ('post', 'responder') 显著差异基因数： 230
('pre', 'nonresponder') ('post', 'nonresponder') 显著差异基因数： 291
('pre', 'responder') ('pre', 'nonresponder') 显著差异基因数： 162
('post', 'responder') ('post', 'nonresponder') 显著差异基因数： 182


In [10]:
go_res["Overlap"]

0       7/104
1        3/22
2       6/214
3       7/314
4        4/72
        ...  
842     1/420
843     1/425
844     1/430
845    6/2206
846     1/474
Name: Overlap, Length: 847, dtype: object

In [51]:
go_res

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,GO_Biological_Process_2021,antigen receptor-mediated signaling pathway (G...,9/185,0.000011,0.011589,0,0,7.135155,81.435926,DENND1B;FYB1;PSME4;PLCG2;PDE4B;NFATC2;PTPRJ;FY...
1,GO_Biological_Process_2021,protein phosphorylation (GO:0006468),14/496,0.000023,0.011589,0,0,4.136441,44.205642,CDK17;CAMK1D;CCNT1;DYRK1A;STK39;PRKX;BRAF;SMAD...
2,GO_Biological_Process_2021,T cell receptor signaling pathway (GO:0050852),8/158,0.000026,0.011589,0,0,7.399061,78.152680,DENND1B;FYB1;PSME4;PLCG2;PDE4B;PTPRJ;FYN;CARD11
3,GO_Biological_Process_2021,positive regulation of DNA-templated transcrip...,4/25,0.000034,0.011589,0,0,25.869537,266.167370,SCAF8;CCNT1;CDK13;CDC73
4,GO_Biological_Process_2021,positive regulation of interleukin-2 productio...,4/26,0.000040,0.011589,0,0,24.692403,250.075060,PDE4B;PLCG2;CARD11;RUNX1
...,...,...,...,...,...,...,...,...,...,...
1445,GO_Biological_Process_2021,translation (GO:0006412),1/214,0.802045,0.804264,0,0,0.618742,0.136488,EEF1D
1446,GO_Biological_Process_2021,"RNA splicing, via transesterification reaction...",1/251,0.850662,0.852425,0,0,0.526174,0.085104,YTHDC1
1447,GO_Biological_Process_2021,positive regulation of cell differentiation (G...,1/258,0.858424,0.859610,0,0,0.511660,0.078109,SYAP1
1448,GO_Biological_Process_2021,central nervous system development (GO:0007417),1/268,0.868823,0.869422,0,0,0.492245,0.069218,NR4A2


In [61]:
# 假设 go_res['Genes'] 是一个包含用逗号分隔的基因字符串，如 "GATA1,KLF1,HBB"
go_res_filtered = go_res[go_res['Genes'].apply(lambda x: len(x.split(';')) >= 5)]

# 查看结果
print(go_res_filtered.head(10))


                      Gene_set  \
0   GO_Biological_Process_2021   
1   GO_Biological_Process_2021   
2   GO_Biological_Process_2021   
5   GO_Biological_Process_2021   
6   GO_Biological_Process_2021   
7   GO_Biological_Process_2021   
8   GO_Biological_Process_2021   
9   GO_Biological_Process_2021   
10  GO_Biological_Process_2021   
12  GO_Biological_Process_2021   

                                                 Term  Overlap   P-value  \
0   antigen receptor-mediated signaling pathway (G...    9/185  0.000011   
1                protein phosphorylation (GO:0006468)   14/496  0.000023   
2      T cell receptor signaling pathway (GO:0050852)    8/158  0.000026   
5   regulation of type I interferon production (GO...     6/89  0.000056   
6   negative regulation of transcription, DNA-temp...   19/948  0.000090   
7   regulation of transforming growth factor beta ...    6/100  0.000107   
8   positive regulation of transcription, DNA-temp...  21/1183  0.000205   
9   positive regu

In [62]:
go_res_filtered

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,GO_Biological_Process_2021,antigen receptor-mediated signaling pathway (G...,9/185,0.000011,0.011589,0,0,7.135155,81.435926,DENND1B;FYB1;PSME4;PLCG2;PDE4B;NFATC2;PTPRJ;FY...
1,GO_Biological_Process_2021,protein phosphorylation (GO:0006468),14/496,0.000023,0.011589,0,0,4.136441,44.205642,CDK17;CAMK1D;CCNT1;DYRK1A;STK39;PRKX;BRAF;SMAD...
2,GO_Biological_Process_2021,T cell receptor signaling pathway (GO:0050852),8/158,0.000026,0.011589,0,0,7.399061,78.152680,DENND1B;FYB1;PSME4;PLCG2;PDE4B;PTPRJ;FYN;CARD11
5,GO_Biological_Process_2021,regulation of type I interferon production (GO...,6/89,0.000056,0.013545,0,0,9.923193,97.140797,CREBBP;CRCP;RNF125;IRF1;NLRC5;GAPDH
6,GO_Biological_Process_2021,"negative regulation of transcription, DNA-temp...",19/948,0.000090,0.018648,0,0,2.954001,27.517769,CREBBP;RBFOX2;SEMA4D;USP9X;SFMBT2;FOXN3;CDC73;...
...,...,...,...,...,...,...,...,...,...,...
860,GO_Biological_Process_2021,regulation of cell migration (GO:0030334),5/408,0.192619,0.324387,0,0,1.663986,2.740655,SEMA4D;PRKX;ATM;PTPRJ;SMAD7
862,GO_Biological_Process_2021,regulation of apoptotic process (GO:0042981),8/742,0.194457,0.324851,0,0,1.467245,2.402676,HSPA9;CREBBP;SEMA4D;GADD45B;MADD;ARHGEF18;BRAF...
963,GO_Biological_Process_2021,regulation of intracellular signal transductio...,5/437,0.231341,0.344738,0,0,1.549968,2.268942,PPP1R16B;PLCG2;ARHGEF18;ATM;PPP2R5C
1064,GO_Biological_Process_2021,positive regulation of cell population prolife...,5/474,0.283606,0.386130,0,0,1.424969,1.795701,TGFBR3;MECP2;CNOT6L;CDK13;FOXP1


In [ ]:
# ---- 你关注的 CD8 五个亚型 ----
CD8_SUBTYPES = ["CD8 CTL", "CD8 Mem", "CD8 Naive", "CD8 Ex", "CD8 EM"]

# ========== Step 1: 计算每个 sample 内亚型比例 ==========
def compute_cd8_subtype_fractions(adata, *, 
                                  treats=None, ven_states=None, response_eq=None,
                                  label_treat=None, label_ven=None,
                                  subtypes=CD8_SUBTYPES):
    obs = adata.obs

    # 标签
    if label_treat is None:
        if treats is None: label_treat = "all_treats"
        else: label_treat = "+".join([treats] if isinstance(treats, str) else treats)
    if label_ven is None:
        if ven_states is not None:
            label_ven = "+".join([ven_states] if isinstance(ven_states, str) else ven_states)
        elif response_eq is not None:
            label_ven = "control" if response_eq == 0 else f"response=={response_eq}"
        else:
            label_ven = "all_states"

    # 筛选
    mask = pd.Series(True, index=obs.index)
    if response_eq is not None:
        mask &= (obs["response"] == response_eq)
    if treats is not None:
        mask &= obs["treat"].isin([treats] if isinstance(treats, str) else treats)
    if ven_states is not None:
        mask &= obs["venetoclax"].isin([ven_states] if isinstance(ven_states, str) else ven_states)
    subset = obs[mask]
    cd8_subset = subset[subset["dcellType"].str.startswith("CD8")]

    # 每个 sample 内的亚型 count 和比例
    def _per_sample(df):
        total_cd8 = len(df)
        counts = df["dcellType"].value_counts()
        out = {"total_cd8": int(total_cd8)}
        for st in subtypes:
            c = int(counts.get(st, 0))
            out[f"count_{st}"] = c
            out[f"frac_{st}"]  = (c / total_cd8) if total_cd8 > 0 else np.nan
        return pd.Series(out)

    per_sample = cd8_subset.groupby("sample", observed=True).apply(_per_sample, include_groups=False)

    # 汇总
    row = {
        "treat": label_treat,
        "venetoclax": label_ven,
        "n_samples_with_CD8": per_sample.shape[0],
        "total_CD8_cells": int(per_sample["total_cd8"].sum()) if per_sample.shape[0] > 0 else 0
    }
    for st in subtypes:
        row[f"mean_{st}"] = per_sample[f"frac_{st}"].mean()
        row[f"std_{st}"]  = per_sample[f"frac_{st}"].std(ddof=1)

    # 长表
    long_rows = []
    for sample, r in per_sample.iterrows():
        for st in subtypes:
            long_rows.append({
                "sample": sample,
                "subtype": st,
                "frac": r[f"frac_{st}"],
                "count": int(r[f"count_{st}"]),
                "total_cd8": int(r["total_cd8"]),
                "treat": label_treat,
                "venetoclax": label_ven
            })
    long_df = pd.DataFrame(long_rows)

    return row, long_df


# ========== Step 2: 定义组别 ==========
groups = [
    dict(treats="pre",  ven_states="responder"),
    dict(treats="post", ven_states="responder"),
    dict(treats="pre",  ven_states="nonresponder"),
    dict(treats="post", ven_states="nonresponder"),
    dict(treats=["pre","post"], ven_states="responder",    label_treat="pre+post"),
    dict(treats=["pre","post"], ven_states="nonresponder", label_treat="pre+post"),
    dict(treats=["pre","post"], ven_states=["responder","nonresponder"],
         label_treat="pre+post", label_ven="responder+nonresponder"),
    dict(response_eq=0, label_treat="healthy", label_ven="control"),
]

rows, long_parts = [], []
for g in groups:
    row, long_df = compute_cd8_subtype_fractions(adata, **g)
    rows.append(row)
    long_parts.append(long_df)

wide_summary = pd.DataFrame(rows)
long_df = pd.concat(long_parts, ignore_index=True)

# 保存
wide_summary.to_csv("CD8_subtype_wide_summary.csv", index=False)
long_df.to_csv("CD8_subtype_per_sample_long.csv", index=False)


# ========== Step 3: 组间统计学比较 ==========
def test_two_groups(df, g1, g2, subtype, method="mannwhitney"):
    a = df[(df["treat"]==g1[0]) & (df["venetoclax"]==g1[1]) & (df["subtype"]==subtype)]["frac"].dropna().values
    b = df[(df["treat"]==g2[0]) & (df["venetoclax"]==g2[1]) & (df["subtype"]==subtype)]["frac"].dropna().values
    label1 = f"{g1[0]}-{g1[1]}"
    label2 = f"{g2[0]}-{g2[1]}"

    if len(a)==0 or len(b)==0:
        return {"subtype":subtype,"group1": label1, "group2": label2, "method": method,
                "n1": len(a), "n2": len(b),
                "mean1": np.nan, "mean2": np.nan, "p": np.nan}

    if method=="mannwhitney":
        stat, p = mannwhitneyu(a, b, alternative="two-sided")
        m = "Mann-Whitney U"
    else:
        stat, p = ttest_ind(a, b, equal_var=False)
        m = "Welch t-test"

    return {"subtype":subtype,"group1": label1, "group2": label2, "method": m,
            "n1": len(a), "n2": len(b),
            "mean1": np.mean(a), "mean2": np.mean(b), "p": p}


def fdr_bh(p):
    p = np.asarray(p, dtype=float)
    n = p.size
    order = np.argsort(p)
    ranked = np.empty_like(order)
    ranked[order] = np.arange(1, n+1)
    q = p * n / ranked
    q_sorted = np.minimum.accumulate(q[order][::-1])[::-1]
    q_adj = np.empty_like(q)
    q_adj[order] = np.minimum(q_sorted, 1.0)
    return q_adj


pairs = [
    (("pre","responder"), ("post","responder")),
    (("pre","nonresponder"), ("post","nonresponder")),
    (("pre","responder"), ("pre","nonresponder")),
    (("post","responder"), ("post","nonresponder")),
    (("pre+post","responder"), ("pre+post","nonresponder")),
    (("pre","responder"), ("healthy","control")),
    (("pre","nonresponder"), ("healthy","control")),
    (("post","responder"), ("healthy","control")),
    (("post","nonresponder"), ("healthy","control")),
    (("pre+post","responder"), ("healthy","control")),
    (("pre+post","nonresponder"), ("healthy","control")),
    (("pre+post","responder+nonresponder"), ("healthy","control")),
]

rows = []
for g1,g2 in pairs:
    for st in CD8_SUBTYPES:
        rows.append(test_two_groups(long_df, g1, g2, st, method="mannwhitney"))
        rows.append(test_two_groups(long_df, g1, g2, st, method="ttest"))

comp = pd.DataFrame(rows)
comp["p_adj_BH"] = fdr_bh(comp["p"].fillna(1.0).values)

comp.to_csv("CD8_pairwise_comparisons.csv", index=False)

print("分析完成：")
print("  - CD8_subtype_wide_summary.csv")
print("  - CD8_subtype_per_sample_long.csv")
print("  - CD8_pairwise_comparisons.csv")


分析完成：
  - CD8_subtype_wide_summary.csv
  - CD8_subtype_per_sample_long.csv
  - CD8_pairwise_comparisons.csv
